In [44]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [45]:

from google.colab import drive
drive.mount('/content/drive')

# train.csvとtest.csvが格納されているディレクトリを指定
target_directory = '/content/drive/MyDrive/kaggle/titanic'

# 作業ディレクトリを変更
os.chdir(target_directory)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
display(df_train.head())
display(df_test.head())
# display(df_train.isnull().sum())
# display(df_test.isnull().sum())
# Ageとcabinに欠損値を確認

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [47]:
# ===== ① train：Pclass別の中央値で補完 =====

df_train["Age"] = df_train.groupby("Pclass")["Age"].transform(
    lambda x: x.fillna(x.median())
)

df_train["Fare"] = df_train.groupby("Pclass")["Fare"].transform(
    lambda x: x.fillna(x.median())
)

# ===== ② test：trainのPclass別中央値を使って補完 =====

for pclass in df_train["Pclass"].unique():
    age_median = df_train[df_train["Pclass"] == pclass]["Age"].median()
    fare_median = df_train[df_train["Pclass"] == pclass]["Fare"].median()

    df_test.loc[(df_test["Pclass"] == pclass) & (df_test["Age"].isnull()), "Age"] = age_median
    df_test.loc[(df_test["Pclass"] == pclass) & (df_test["Fare"].isnull()), "Fare"] = fare_median

# print("train 欠損数")
# print(df_train[["Age", "Fare"]].isnull().sum())

# print("test 欠損数")
# print(df_test[["Age", "Fare"]].isnull().sum())

In [48]:
#Familysize
# train
df_train["FamilySize"] = df_train["SibSp"] + df_train["Parch"] + 1
# test
df_test["FamilySize"] = df_test["SibSp"] + df_test["Parch"] + 1

#Isalone
# train
df_train["IsAlone"] = ((df_train["SibSp"] == 0) & (df_train["Parch"] == 0)).astype(int)
# test
df_test["IsAlone"] = ((df_test["SibSp"] == 0) & (df_test["Parch"] == 0)).astype(int)


# train
df_train["IsCouple"] = ((df_train["SibSp"] == 1) & (df_train["Parch"] == 0)).astype(int)

# test
df_test["IsCouple"] = ((df_test["SibSp"] == 1) & (df_test["Parch"] == 0)).astype(int)

# Title 抽出
df_train["Title"] = df_train["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
df_test["Title"] = df_test["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)

# レアタイトルを統合
mapping = {
    "Lady": "Rare", "Countess": "Rare", "Capt": "Rare",
    "Col": "Rare", "Don": "Rare", "Dr": "Rare",
    "Major": "Rare", "Rev": "Rare", "Sir": "Rare",
    "Jonkheer": "Rare", "Dona": "Rare",
    "Mlle": "Miss", "Ms": "Miss",
    "Mme": "Mrs"
}

df_train["Title"] = df_train["Title"].replace(mapping)
df_test["Title"]  = df_test["Title"].replace(mapping)




In [49]:
from IPython.lib.security import random
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
# !pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# display(df_train.head())



train_survived = df_train["Survived"]

features = ["Pclass","Sex","Age","Fare","Embarked","FamilySize","IsAlone", "IsCouple","Title"]




train_feature = pd.get_dummies(df_train[features])
test_feature = pd.get_dummies(df_test[features])




model = lgb.LGBMClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=5,
    num_leaves=15,
    min_data_in_leaf=40,
    random_state=1
)


model.fit(train_feature, train_survived)
predictions = model.predict(test_feature)


output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': predictions})
output.to_csv('./result_csv/submission_ver10.csv', index=False)
print("Your submission was successfully saved!")

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000138 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 234
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga